In [95]:
import sys
import os
import json
import datetime
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf
import predictions as pr
import cost

def read_in():
    if sys.stdin.isatty()== True:
        return({'startdate':datetime.date.today(),'year':5,'type':'month','size':99,'weight':1.5,'average_time':3.4})
    if sys.stdin.isatty()== False:
        lines=sys.stdin.readlines()
        parsed=json.loads(lines[0])
        return(parsed)
params={'startdate':'2018-05-01','year':15,'size':99.5,'weight':1.5,'averagetime':3.4,"scenario":"lm_model","type":"month"}

In [96]:
params['year']=int(params['year']);params['size']=float(params['size']);params['weight']=float(params['weight']);params['averagetime']=float(params['averagetime']);

index_month=pd.period_range(start=params["startdate"],periods=params["year"]*12+1,freq="m")
index_quarter=pd.period_range(start=params["startdate"],periods=params["year"]*4+1,freq="q")
index_year=pd.period_range(start=params["startdate"],periods=params["year"]+1,freq="y")

params['year']=int(params['year']);params['size']=float(params['size']);params['weight']=float(params['weight']);params['averagetime']=float(params['averagetime']);
cost=cost.total_cost_calc(params)
revenue=pr.predict(params['scenario'],params['startdate'],params['year'])
revenue.index=index_month;
revenue['days']=revenue.index.day
revenue['rec_price']=100
revenue['smp_revenue']=revenue['smp_price']*30*params['weight']*params['size']*params['averagetime']
revenue['rec_revenue']=revenue['rec_price']*30*params['weight']*params['size']*params['averagetime']
revenue.index=index_month
result=pd.concat([revenue,cost],axis=1)

In [97]:
start=pd.to_datetime(params["startdate"], format='%Y-%m-%d', errors='ignore')
startday=start.date().timetuple()

result.days[0]=result.days[0]-startday.tm_mday
result.days[result.shape[0]-1]=startday.tm_mday
result["date"]=index_month
result["smp_revenue"]=result["smp_revenue"].astype(int)
result["rec_revenue"]=result["rec_revenue"].astype(int)

price_forecast=result.loc[:,['smp_price','rec_price']]
price_forecast.index=index_month
price_forecast.index

/Users/youngji/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/youngji/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


PeriodIndex(['2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10',
             '2018-11', '2018-12', '2019-01', '2019-02',
             ...
             '2032-08', '2032-09', '2032-10', '2032-11', '2032-12', '2033-01',
             '2033-02', '2033-03', '2033-04', '2033-05'],
            dtype='period[M]', length=181, freq='M')

In [98]:
result_sample=result.drop(['rec_price','smp_price'],axis=1)
#월간 표
result_month=result_sample
result_month.index=index_month
result_month.index

#분기 표
def toQuarter(data):
    quarter= []
    for i in range(0,data.index.size):
        quarter.append(str(data.index.year[i])+'-'+str(data.index.quarter[i]))
    return quarter
result_quarter=result_sample
result_quarter.index=index_month
result_quarter.index=toQuarter(result_quarter)
result_quarter=result_quarter.groupby(result_quarter.index).sum()
result_quarter.index=index_quarter

#연간 가격 
result_year=result_sample
result_year.index=index_month
result_year.index=result_year.index.year
result_year=result_year.groupby(result_year.index).sum()
result_year.index=index_year
result_year

,days,smp_revenue,rec_revenue,machine_cost,monitoring_cost,insurance_cost,reserve_cost,elec_safety_cost,operation_cost,account_cost,office_cost,total_cost
2018,244,10291084,12178800,792,4776,4776,2384,955200,183080,796000,1830800,3777808
2019,365,15330061,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2020,366,15999896,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2021,365,15954224,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2022,365,15056041,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2023,365,15132156,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2024,366,16091236,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2025,365,17126434,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2026,365,17583141,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
2027,365,17720148,18268200,1188,7164,7164,3576,1432800,274620,1194000,2746200,5666712
